This notebook demonstrates the use of Robert Kern's `line_profiler` (https://github.com/rkern/line_profiler) to profile `cython` functions line by line. Thanks to Robert Bradshaw for implementing this feature (https://groups.google.com/forum/#!topic/cython-users/FH3TYK8BkeA).

In [2]:
#Load Robert Kern's line profiler
%load_ext line_profiler
%load_ext Cython
import line_profiler

In [3]:
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
import Cython
directive_defaults = Cython.Compiler.Options.get_directive_defaults()

directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [5]:
import numpy as np
import pickle

In [28]:
with open('cython_test_case.pickle', 'rb') as fp:
    first, class_to_neighbor_dict, pixel_to_row, neighbors, districts = pickle.load(fp)

In [29]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
#We need to define the macro CYTHON_TRACE=1 (cf. http://docs.cython.org/src/reference/compilation.html)

cimport numpy as cnp
cnp.import_array()

def int_in_list(int target, list to_search):
    for idx in range(len(to_search)):
        possible = to_search[idx]
        if possible == target:
            return True
        
    return False

def district_breaks(int swap_pixel, dict to_search, dict pix_num_to_row, cnp.ndarray[long, ndim=2] neighbors,
                    cnp.ndarray[long, ndim=1] districts):
    cdef list queue, neighbor_list, visited
    cdef cnp.ndarray[long, ndim=1] focus_neighbors
    cdef int class_num
    cdef int start, focus, focus_neighbor

    for class_num, neighbor_list in to_search.items():
        start = neighbor_list.pop()
        visited = [start]
        queue = [start]

        while queue:
            focus = queue.pop(0)
            if focus == -1:
                continue
            if focus in neighbor_list:
                neighbor_list.remove(focus)
                if not neighbor_list:
                    break

            focus_neighbors = neighbors[pix_num_to_row[focus]]
            for idx in range(len(focus_neighbors)):
                focus_neighbor = focus_neighbors[idx]
                if focus_neighbor == -1:
                    continue
                if (
                    focus_neighbor not in visited and
                    districts[pix_num_to_row[focus_neighbor]] == class_num and
                    focus_neighbor != swap_pixel
                ):
                    visited.append(focus_neighbor)
                    queue.append(focus_neighbor)

        if neighbor_list:
            return True

    return False

In file included from /Users/cbitting/.cache/ipython/cython/_cython_magic_621312ef4a056a1935952834c5b4b8c2.c:715:
In file included from /Users/cbitting/miniconda3/envs/gerrymandering/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /Users/cbitting/miniconda3/envs/gerrymandering/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/cbitting/miniconda3/envs/gerrymandering/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1960:
/Users/cbitting/miniconda3/envs/gerrymandering/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
/Users/cbitting/.cache/ipython/cython/_cython_magic_621312ef4a056a1935952834c5b4b8c2.c:7928:5: warning: 'tp_print' is deprecated [-Wdepreca

In [30]:
#Print profiling statistics using the `line_profiler` API
profile = line_profiler.LineProfiler(district_breaks)
profile.runcall(district_breaks, first, class_to_neighbor_dict, pixel_to_row, neighbors, districts)
profile.print_stats()

Timer unit: 1e-06 s

Total time: 0.001633 s
File: /Users/cbitting/.cache/ipython/cython/_cython_magic_621312ef4a056a1935952834c5b4b8c2.pyx
Function: district_breaks at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                           def district_breaks(int swap_pixel, dict to_search, dict pix_num_to_row, cnp.ndarray[long, ndim=2] neighbors,
    15                                                               cnp.ndarray[long, ndim=1] districts):
    16                                               cdef list queue, neighbor_list, visited
    17                                               cdef cnp.ndarray[long, ndim=1] focus_neighbors
    18                                               cdef int class_num
    19                                               cdef int start, focus, focus_neighbor
    20                                           
    21         1          2.0      2.0      0.1      for class_num, neighbor_list in to_

In [13]:
#Print profiling statistics using the `lprun` magic
%lprun -f district_breaks district_breaks(first, class_to_neighbor_dict, pixel_to_row, neighbors, districts)

IndexError: pop from empty list